# Notebook: Filter Sentences from Reviews to create Dataset for Annotation

## Packages

In [1]:
from bs4 import BeautifulSoup
from langdetect import detect, DetectorFactory
import pandas as pd
import nltk
import uuid
from nltk.tokenize import sent_tokenize

## Constants

In [2]:
REVIEWS_PATH = "reviews_dataset/reviews.csv"
RANDOM_STATE = 43
LOAD_1000_ADDITIONAL_SENTENCES = True

## Settings

In [3]:
DetectorFactory.seed = RANDOM_STATE

In [4]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/nils_hellwig/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Code

### Load Dataset

In [5]:
df_reviews = pd.read_csv(REVIEWS_PATH)

In [6]:
len(df_reviews)

3212

In [7]:
# Höchstes Datum
max_date = df_reviews['date'].max()

# Niedrigstes Datum
min_date = df_reviews['date'].min()

print("Höchstes Datum:", max_date)
print("Niedrigstes Datum:", min_date)

Höchstes Datum: 2023-10-15
Niedrigstes Datum: 2022-10-15


### Filter Sentences

In [8]:
df_reviews_sentences = pd.DataFrame(columns=list(df_reviews.columns) + ['sentence_idx', "id"])

for idx, row in df_reviews.iterrows():
    sentences = sent_tokenize(row['text'], language="german")
    sentence_index = 0
    for sentence in sentences:
        if len(sentence) > 1:
            new_row = row.copy() 
            new_row['text'] = sentence 
            new_row['id'] = str(uuid.uuid4())
            new_row['sentence_idx'] = sentence_index
            df_reviews_sentences.loc[len(df_reviews_sentences)] = new_row
            sentence_index += 1

In [9]:
df_reviews_sentences

,Unnamed: 0,review_id,restaurant_id,page_index,title,date,author_name,author_location,text,rating,city,restaurant_name,language_code,text_noanonymization,sentence_idx,id
0,0,921348897,2005104,0,Bayrisches Bier mitten in Berlin,2023-10-15,wolfganga300,Erlangen,Bayrisches Bier mitten in LOC.Das kann man im ...,5.0,berlin,Augustiner am Gendarmenmarkt,de,Bayrisches Bier mitten in Berlin.Das kann man ...,0,100b2ebe-de7d-47e2-8a84-b621f6b5f84e
1,0,921348897,2005104,0,Bayrisches Bier mitten in Berlin,2023-10-15,wolfganga300,Erlangen,"Das Lokal ist immer voll, eine Reservierung is...",5.0,berlin,Augustiner am Gendarmenmarkt,de,Bayrisches Bier mitten in Berlin.Das kann man ...,1,cf382cc1-ee04-429d-86eb-ec56d6b7def6
2,0,921348897,2005104,0,Bayrisches Bier mitten in Berlin,2023-10-15,wolfganga300,Erlangen,Innen ist es gemütlich aber auch laut.,5.0,berlin,Augustiner am Gendarmenmarkt,de,Bayrisches Bier mitten in Berlin.Das kann man ...,2,c6dbc833-4686-46f0-9a3b-4f1210fc5959
3,0,921348897,2005104,0,Bayrisches Bier mitten in Berlin,2023-10-15,wolfganga300,Erlangen,Ein Teil des Personals versucht sich südbayeri...,5.0,berlin,Augustiner am Gendarmenmarkt,de,Bayrisches Bier mitten in Berlin.Das kann man ...,3,d670caaa-430c-4a0f-83cd-c900cba7dec5
4,1,921129385,2005104,0,Denn Sie wissen nicht was Sie tun!!!,2023-10-13,Maximilian-62,"Freiburg, Deutschland",Bayrisch Essen und Gastfreundschaft geht anders.,2.0,berlin,Augustiner am Gendarmenmarkt,de,Bayrisch Essen und Gastfreundschaft geht ander...,0,2fb56328-f1b5-4d69-a8e2-5b5cd378756c
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13421,3211,865168700,5207419,0,Ich liebe diese Haxe,2022-10-19,BibiH41366,NaN,Haxe wo man noch nicht mal ein Messer braucht.,5.0,mönchengladbach,Brauerei zum Stefanus,de,Wir sind nicht zum ersten Mal aber auch defini...,1,d3072d38-99a9-47d1-afef-f970b26225d0
13422,3211,865168700,5207419,0,Ich liebe diese Haxe,2022-10-19,BibiH41366,NaN,Und ich weiß dass ich niemals irgendwo anders ...,5.0,mönchengladbach,Brauerei zum Stefanus,de,Wir sind nicht zum ersten Mal aber auch defini...,2,709d6b9c-9565-46fb-8b34-b442e186b7a7
13423,3211,865168700,5207419,0,Ich liebe diese Haxe,2022-10-19,BibiH41366,NaN,Super nettes aufmerksames Personal.,5.0,mönchengladbach,Brauerei zum Stefanus,de,Wir sind nicht zum ersten Mal aber auch defini...,3,cc7adc68-1cf8-4ee1-89a0-c2fe3d8f900c
13424,3211,865168700,5207419,0,Ich liebe diese Haxe,2022-10-19,BibiH41366,NaN,Chef kommt auch selber an den Tisch und ist se...,5.0,mönchengladbach,Brauerei zum Stefanus,de,Wir sind nicht zum ersten Mal aber auch defini...,4,e384b31b-c2b5-4e00-8b9c-7c4c688e93e7


### Filter Non-German Sentences

Even if the language code = "de", we have observed that some sentences are in other languagesthan german. These are excluded as well with the help of Google's language-detection.

In [10]:
def detect_language(text):
    try:
        return detect(text)
    except:
        return 'Unknown'

df_reviews_sentences['detected_language'] = df_reviews_sentences['text'].apply(detect_language)
df_reviews_sentences[df_reviews_sentences['detected_language'] != 'de']

,Unnamed: 0,review_id,restaurant_id,page_index,title,date,author_name,author_location,text,rating,city,restaurant_name,language_code,text_noanonymization,sentence_idx,id,detected_language
21,3,918788541,2005104,0,Nicht zu empfehlen! Zero Dienstleistungsverstä...,2023-09-28,cariocadagema74,"Berlin, Deutschland",Kein Cent!,1.0,berlin,Augustiner am Gendarmenmarkt,de,Dieser Laden macht den Ruhm Berlins als Servic...,10,18a272b6-f27a-4095-a5ef-16c73b9ccfef,nl
54,9,890917149,2005104,0,schlecht verwaltetes Chaos,2023-05-18,Harald F,"Berlin, Deutschland",Es war viel los.,1.0,berlin,Augustiner am Gendarmenmarkt,de,ich hätte gerne etwas zum Essen gesagt. Aber e...,3,b5b595e2-51d6-4020-92d9-931e60bb9bb0,es
103,18,866452621,2005104,1,Mind this touri-gap,2022-10-29,452stephand,"Nürnberg, Deutschland",Fazit: mind this touri-gap,1.0,berlin,Augustiner am Gendarmenmarkt,de,Wunderschöne Location mit mangelhafter Leitung...,1,388164d1-5f45-4222-8976-203c59697ee1,en
111,21,883653345,947082,0,Tolle Burger!,2023-03-26,dittelbrunn,"Lübeck, Deutschland",Ketchup.,5.0,berlin,Burgermeister,de,Wir waren in der Filiale am S-Bahnhof Eberswal...,2,f6fa05fb-384c-49e5-8564-1021b7b4a09b,id
115,22,883285244,947082,0,Ganz gut,2023-03-23,Mawenabert,NaN,Ansonsten alles gut,4.0,berlin,Burgermeister,de,Hatte einen Hausmeister Burger mit Pommes Baco...,2,9192a562-f082-4c4b-98cb-ce6afb3ceae8,fr
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13318,3180,873342513,13144674,5,Wie immer super,2022-12-30,nelala79,NaN,Service super.,5.0,mönchengladbach,Frenzen Restaurant & Event,de,Salat mit Genüsenuggets gegessen. Wie man es h...,2,cb5c24ab-527e-4f2a-ae4c-ed3093d9fd95,en
13319,3180,873342513,13144674,5,Wie immer super,2022-12-30,nelala79,NaN,Essen super.,5.0,mönchengladbach,Frenzen Restaurant & Event,de,Salat mit Genüsenuggets gegessen. Wie man es h...,3,528cef39-5e14-4e7e-8eb2-b4207a0f2e8b,ca
13322,3180,873342513,13144674,5,Wie immer super,2022-12-30,nelala79,NaN,Klar!,5.0,mönchengladbach,Frenzen Restaurant & Event,de,Salat mit Genüsenuggets gegessen. Wie man es h...,6,1d1c4853-3897-4c66-ac8d-25f506df7956,tr
13356,3195,866578862,13144674,6,Leckeres Essen in Mönchengladbach,2022-10-30,marcelbF2818DO,NaN,Ein tolles Restaurant!,5.0,mönchengladbach,Frenzen Restaurant & Event,de,Ein tolles Restaurant! Ambiente und Empfang su...,0,29fd2d1d-9160-49cc-90f6-42e824819642,fr


In [11]:
df_reviews_sentences = df_reviews_sentences[df_reviews_sentences['detected_language'] == 'de']

In [12]:
df_reviews_sentences

,Unnamed: 0,review_id,restaurant_id,page_index,title,date,author_name,author_location,text,rating,city,restaurant_name,language_code,text_noanonymization,sentence_idx,id,detected_language
0,0,921348897,2005104,0,Bayrisches Bier mitten in Berlin,2023-10-15,wolfganga300,Erlangen,Bayrisches Bier mitten in LOC.Das kann man im ...,5.0,berlin,Augustiner am Gendarmenmarkt,de,Bayrisches Bier mitten in Berlin.Das kann man ...,0,100b2ebe-de7d-47e2-8a84-b621f6b5f84e,de
1,0,921348897,2005104,0,Bayrisches Bier mitten in Berlin,2023-10-15,wolfganga300,Erlangen,"Das Lokal ist immer voll, eine Reservierung is...",5.0,berlin,Augustiner am Gendarmenmarkt,de,Bayrisches Bier mitten in Berlin.Das kann man ...,1,cf382cc1-ee04-429d-86eb-ec56d6b7def6,de
2,0,921348897,2005104,0,Bayrisches Bier mitten in Berlin,2023-10-15,wolfganga300,Erlangen,Innen ist es gemütlich aber auch laut.,5.0,berlin,Augustiner am Gendarmenmarkt,de,Bayrisches Bier mitten in Berlin.Das kann man ...,2,c6dbc833-4686-46f0-9a3b-4f1210fc5959,de
3,0,921348897,2005104,0,Bayrisches Bier mitten in Berlin,2023-10-15,wolfganga300,Erlangen,Ein Teil des Personals versucht sich südbayeri...,5.0,berlin,Augustiner am Gendarmenmarkt,de,Bayrisches Bier mitten in Berlin.Das kann man ...,3,d670caaa-430c-4a0f-83cd-c900cba7dec5,de
4,1,921129385,2005104,0,Denn Sie wissen nicht was Sie tun!!!,2023-10-13,Maximilian-62,"Freiburg, Deutschland",Bayrisch Essen und Gastfreundschaft geht anders.,2.0,berlin,Augustiner am Gendarmenmarkt,de,Bayrisch Essen und Gastfreundschaft geht ander...,0,2fb56328-f1b5-4d69-a8e2-5b5cd378756c,de
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13421,3211,865168700,5207419,0,Ich liebe diese Haxe,2022-10-19,BibiH41366,NaN,Haxe wo man noch nicht mal ein Messer braucht.,5.0,mönchengladbach,Brauerei zum Stefanus,de,Wir sind nicht zum ersten Mal aber auch defini...,1,d3072d38-99a9-47d1-afef-f970b26225d0,de
13422,3211,865168700,5207419,0,Ich liebe diese Haxe,2022-10-19,BibiH41366,NaN,Und ich weiß dass ich niemals irgendwo anders ...,5.0,mönchengladbach,Brauerei zum Stefanus,de,Wir sind nicht zum ersten Mal aber auch defini...,2,709d6b9c-9565-46fb-8b34-b442e186b7a7,de
13423,3211,865168700,5207419,0,Ich liebe diese Haxe,2022-10-19,BibiH41366,NaN,Super nettes aufmerksames Personal.,5.0,mönchengladbach,Brauerei zum Stefanus,de,Wir sind nicht zum ersten Mal aber auch defini...,3,cc7adc68-1cf8-4ee1-89a0-c2fe3d8f900c,de
13424,3211,865168700,5207419,0,Ich liebe diese Haxe,2022-10-19,BibiH41366,NaN,Chef kommt auch selber an den Tisch und ist se...,5.0,mönchengladbach,Brauerei zum Stefanus,de,Wir sind nicht zum ersten Mal aber auch defini...,4,e384b31b-c2b5-4e00-8b9c-7c4c688e93e7,de


### Balancing

In [13]:
existing_sentences_path = 'reviews_dataset/balanced_reviews_sentences_4000.csv'

if LOAD_1000_ADDITIONAL_SENTENCES:
    df_4000 = pd.read_csv(existing_sentences_path)
    df_reviews_sentences = df_reviews_sentences[~df_reviews_sentences.set_index(['review_id', 'restaurant_id', 'sentence_idx']).index.isin(
    df_reviews_sentences.set_index(['review_id', 'restaurant_id', 'sentence_idx']).index.intersection(
        df_4000.set_index(['review_id', 'restaurant_id', 'sentence_idx']).index
      )
    )]

In [15]:
df_balanced_reviews_sentences = df_reviews_sentences.groupby(['rating'], group_keys=False).apply(lambda x: x.sample(200, replace=False, random_state=RANDOM_STATE))
df_balanced_reviews_sentences.reset_index(drop=True, inplace=True)

### Randomisierung

In [16]:
df_balanced_reviews_sentences = df_balanced_reviews_sentences.sample(frac=1, random_state=RANDOM_STATE)  # 'frac=1' mischt den gesamten DataFrame
df_balanced_reviews_sentences.reset_index(drop=True, inplace=True)

### Check If Sentences are Unique

In [17]:
df_balanced_reviews_sentences.groupby(["review_id", "sentence_idx"]).size().reset_index(name="count")

,review_id,sentence_idx,count
0,864600780,0,1
1,864603326,5,1
2,864603326,6,1
3,864686123,1,1
4,864747211,4,1
...,...,...,...
995,921404419,0,1
996,921404419,1,1
997,921404419,2,1
998,921404419,3,1


### Alternate examples based on rating

In [18]:
df_balanced_reviews_sentences["rating"].value_counts()

5.0    200
1.0    200
3.0    200
4.0    200
2.0    200
Name: rating, dtype: int64

In [19]:
dfs = [df_balanced_reviews_sentences[df_balanced_reviews_sentences["rating"] == i] for i in range(1, 6)]
df_alternating = pd.DataFrame(columns=df_balanced_reviews_sentences.columns)

max_rows = max(len(df) for df in dfs)
for i in range(max_rows):
    for k in range(5):
        if i < len(dfs[k]):
            df_alternating = pd.concat([df_alternating, dfs[k].iloc[[i]]])

df_balanced_reviews_sentences = df_alternating.reset_index(drop=True)

In [20]:
df_balanced_reviews_sentences

,Unnamed: 0,review_id,restaurant_id,page_index,title,date,author_name,author_location,text,rating,city,restaurant_name,language_code,text_noanonymization,sentence_idx,id,detected_language
0,2012,911767426,782302,0,Pech gehabt!,2023-08-20,pittinicki,"Luxemburg, Luxemburg","Die Gäste, die dann noch im Restaurant sind, h...",1.0,dresden,Pulverturm an der Frauenkirche,de,Wir waren gegen 20.00 Uhr im Restaurant. Man l...,4,7a8af706-b7e9-40db-80ca-03fb392df7d6,de
1,2204,880730953,1504431,0,Menükarussel der anderen Art,2023-03-02,B5067QCmartinas,NaN,Schade!,2.0,bochum,Yamas meze restaurant & weinbar,de,Optisch ein sehr schönes Lokal. Leider war die...,8,0eee5bde-a56e-490c-865e-ee38b5b68b02,de
2,3015,880194942,2640161,0,Essen Flop👎 Ambiente/Service/Bier Top👍,2023-02-26,Vegasmann,"Rosenheim, Deutschland",Das Ambiente ist auch super und für ein Bräust...,3.0,augsburg,Riegele Wirtshaus,de,Wir waren zum Abendessen im Riegele Wirtshaus....,2,92ceb6b4-7d93-4b9f-a08c-645318d605a4,de
3,435,879355767,10322027,18,Authentisch!,2023-02-19,Sightsee42202584629,"La Gomera, Spanien",Hier waren wir nach einem Konzert in der LOC u...,4.0,hamburg,Ola Lisboa,de,Es gibt so viele Restaurants im Portugiesenvie...,1,8bc6666b-9a3c-408c-86dc-a56726e9a22a,de
4,3211,865168700,5207419,0,Ich liebe diese Haxe,2022-10-19,BibiH41366,NaN,Und ich weiß dass ich niemals irgendwo anders ...,5.0,mönchengladbach,Brauerei zum Stefanus,de,Wir sind nicht zum ersten Mal aber auch defini...,2,709d6b9c-9565-46fb-8b34-b442e186b7a7,de
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2231,890623499,1981670,0,Nie wieder,2023-05-16,Dirk T,NaN,Hatten ein krüstchen bestellt.,1.0,wuppertal,Wuppertaler Brauhaus,de,Nie wieder. Hatten ein krüstchen bestellt. Der...,1,814393d4-d43c-42e4-b5f3-154632515082,de
996,81,910530241,1483463,2,Schade,2023-08-15,Schneeflockeelse,"Hamburg, Deutschland",Ziemliche Frechheit.,2.0,hamburg,Bullerei,de,Wir waren jahrelang Stammgast im Aussenbereich...,3,a4f65da8-d2a3-4d01-9094-bf85768abb73,de
997,3045,914446690,1005252,0,Einer der schönsten Aussenplätze in Wiesbaden...,2023-09-02,CallPat,"Wiesbaden, Deutschland",Fast alles in Ordnung.,3.0,wiesbaden,BENNER´s Bistronomie,de,Vor einem Konzert im Kurhaus haben wir uns auf...,6,5d36a330-fa4f-4108-8f64-dc6677dc966f,de
998,614,915377469,807825,0,Schöner Biergartrn,2023-09-07,298stefaniem,"Valencia, Spanien",Fleissiger Service.,4.0,münchen,Augustiner-Keller,de,Netter und grosser Biergaeten. Man bekommt eig...,2,345ae479-581c-4e66-ad67-44121f542f45,de


### Store as .csv 

In [21]:
if LOAD_1000_ADDITIONAL_SENTENCES == False:
   df_reviews_sentences.to_csv("reviews_dataset/reviews_sentences.csv")

In [22]:
df_balanced_reviews_sentences.to_csv("reviews_dataset/balanced_reviews_sentences.csv")

In [23]:
df_balanced_reviews_sentences

,Unnamed: 0,review_id,restaurant_id,page_index,title,date,author_name,author_location,text,rating,city,restaurant_name,language_code,text_noanonymization,sentence_idx,id,detected_language
0,2012,911767426,782302,0,Pech gehabt!,2023-08-20,pittinicki,"Luxemburg, Luxemburg","Die Gäste, die dann noch im Restaurant sind, h...",1.0,dresden,Pulverturm an der Frauenkirche,de,Wir waren gegen 20.00 Uhr im Restaurant. Man l...,4,7a8af706-b7e9-40db-80ca-03fb392df7d6,de
1,2204,880730953,1504431,0,Menükarussel der anderen Art,2023-03-02,B5067QCmartinas,NaN,Schade!,2.0,bochum,Yamas meze restaurant & weinbar,de,Optisch ein sehr schönes Lokal. Leider war die...,8,0eee5bde-a56e-490c-865e-ee38b5b68b02,de
2,3015,880194942,2640161,0,Essen Flop👎 Ambiente/Service/Bier Top👍,2023-02-26,Vegasmann,"Rosenheim, Deutschland",Das Ambiente ist auch super und für ein Bräust...,3.0,augsburg,Riegele Wirtshaus,de,Wir waren zum Abendessen im Riegele Wirtshaus....,2,92ceb6b4-7d93-4b9f-a08c-645318d605a4,de
3,435,879355767,10322027,18,Authentisch!,2023-02-19,Sightsee42202584629,"La Gomera, Spanien",Hier waren wir nach einem Konzert in der LOC u...,4.0,hamburg,Ola Lisboa,de,Es gibt so viele Restaurants im Portugiesenvie...,1,8bc6666b-9a3c-408c-86dc-a56726e9a22a,de
4,3211,865168700,5207419,0,Ich liebe diese Haxe,2022-10-19,BibiH41366,NaN,Und ich weiß dass ich niemals irgendwo anders ...,5.0,mönchengladbach,Brauerei zum Stefanus,de,Wir sind nicht zum ersten Mal aber auch defini...,2,709d6b9c-9565-46fb-8b34-b442e186b7a7,de
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2231,890623499,1981670,0,Nie wieder,2023-05-16,Dirk T,NaN,Hatten ein krüstchen bestellt.,1.0,wuppertal,Wuppertaler Brauhaus,de,Nie wieder. Hatten ein krüstchen bestellt. Der...,1,814393d4-d43c-42e4-b5f3-154632515082,de
996,81,910530241,1483463,2,Schade,2023-08-15,Schneeflockeelse,"Hamburg, Deutschland",Ziemliche Frechheit.,2.0,hamburg,Bullerei,de,Wir waren jahrelang Stammgast im Aussenbereich...,3,a4f65da8-d2a3-4d01-9094-bf85768abb73,de
997,3045,914446690,1005252,0,Einer der schönsten Aussenplätze in Wiesbaden...,2023-09-02,CallPat,"Wiesbaden, Deutschland",Fast alles in Ordnung.,3.0,wiesbaden,BENNER´s Bistronomie,de,Vor einem Konzert im Kurhaus haben wir uns auf...,6,5d36a330-fa4f-4108-8f64-dc6677dc966f,de
998,614,915377469,807825,0,Schöner Biergartrn,2023-09-07,298stefaniem,"Valencia, Spanien",Fleissiger Service.,4.0,münchen,Augustiner-Keller,de,Netter und grosser Biergaeten. Man bekommt eig...,2,345ae479-581c-4e66-ad67-44121f542f45,de


In [24]:
df_balanced_reviews_sentences["rating"].value_counts()

1.0    200
2.0    200
3.0    200
4.0    200
5.0    200
Name: rating, dtype: int64